### Import packages

In [3]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
from urllib.request import urlopen
from tqdm import tqdm
import random
import time
from selenium import webdriver

### Global variables

In [4]:
categories = {'sport': 'adventure-and-sport',
             'music': 'music',
             'film': 'film-and-video',
             'theatre': 'live-performance'}

### Functions

In [71]:
# Create an url
def create_url(page, category):
    _filter = '&categories[' + categories[category] + ']=on'
    return 'https://www.kisskissbankbank.com/en/discover?project[successful]=on' + _filter + '&page=' + str(page)

In [79]:
# Get a page from url
def get_page(url):
    ok = False
    page = requests.get(url)
    time.sleep(5)
    page = requests.get(url)
    if(page.status_code == 200):
        ok = True
    bs = BeautifulSoup(page.text, 'html.parser')
    return bs, ok

In [7]:
# Get the number of the projects from bs
def get_num_of_projects(bs):
    total_projects = bs.findAll('span', 'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-color-font1 k-u-size-tiny k-u-weight-regular')
    return int(total_projects[0].contents[0].split(' ')[0].replace(',','')) 

In [8]:
# Get the number of pages from bs
def get_num_of_pages(bs):
    num_pro = get_num_of_projects(bs)
    return int(num_pro/9) + 1

In [83]:
# Function to go to project page and get information
def go_to_project(url_pro):
    
    # Load the website by driver
    chrome_path = './chromedriver.exe'
    driver = webdriver.Chrome(chrome_path)
    driver.get(url_pro)
    time.sleep(5)
    
    # Accept the cookie
    cookie_question = driver.find_element_by_xpath("//*[@id=\"App-react-component\"]/div[1]/div/div/div/div/button[2]")
    cookie_question.click()
    
    # Load the html page
    html = driver.page_source
    bs = BeautifulSoup(html, 'html.parser')
    
    # Calculate num_news, num_comments, num_contributions
    infos = bs.findAll('a', 'k-NavBar__link')
    news_bs = infos[2].findAll('span', 'badge__StyledBadge-sc-7liuod-0 eucluz k-Badge k-Badge--spaced')
    comments_bs = infos[3].findAll('span', 'badge__StyledBadge-sc-7liuod-0 eucluz k-Badge k-Badge--spaced')
    contributions_bs = infos[4].findAll('span', 'badge__StyledBadge-sc-7liuod-0 eucluz k-Badge k-Badge--spaced')
    
    if(len(news_bs) == 0):
        num_news = 0
    else:
        num_news = int(news_bs[0].contents[0])

    if(len(comments_bs) == 0):
        num_comments = 0
    else:
        num_comments = int(comments_bs[0].contents[0])

    if(len(contributions_bs) == 0):
        num_contributions = 0
    else:
        num_contributions = int(contributions_bs[0].contents[0])
    
    # Calculate num_rewards, min_price, max_price, and backers each price
    rewards = bs.findAll('div','marger__StyledMarger-sc-1qqifp5-0 kXGmDY')
    prices = []
    backers = []
    for j in range(len(rewards)):
        rj = rewards[j]
        price_j = rj.findAll('h2', 'title__StyledTitle-sc-46lshq-0 edmigd k-RewardCard__title k-u-margin-bottom-double k-u-margin-top-none')
        if(len(price_j) == 0):
            break
        price_j = float(price_j[0].contents[0].replace('€','').replace(',',''))
        prices.append(price_j)
        backers_j = rj.findAll('span', 'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-color-font1 k-u-size-micro k-u-weight-regular k-RewardCard__infos k-RewardCard__infos--hasBottomMargin k-RewardCard__infos--disabled')
        bj = 0
        if(len(backers_j) != 0 and backers_j[0].contents[0].replace(' ','').isnumeric()):
            bj = int(backers_j[0].contents[0])
        backers.append(bj)
    prices_np = np.array(prices)
    backers_np = np.array(backers)
    min_price = np.min(prices_np)
    max_price = np.max(prices_np)
    num_rewards = len(prices_np)
    #print(backers_np)
    
    # Calculate num_creator_projects
    url_owner = bs.findAll('a', 'owner-info__StyledOwnerGrid-tqxc8c-0 jnrkmW')[0].get('href')
    driver_owner = webdriver.Chrome(chrome_path)
    driver_owner.get('https://www.kisskissbankbank.com'+ url_owner)
    time.sleep(5)
    cookie_question_owner = driver_owner.find_element_by_xpath('//*[@id="Header-react-component"]/div[1]/div/div/div/div/button[2]')
    cookie_question_owner.click()
    html_owner = driver_owner.page_source
    bs_owner = BeautifulSoup(html_owner, 'html.parser')
    num_creator_projects = int(bs_owner.findAll('span', 'k-Badge k-HorizontalNav__badge')[0].contents[0])
    #print(num_creator_projects)
    
    driver_owner.quit()
    driver.quit()
    return num_news, num_comments, num_contributions, num_rewards, min_price, max_price, prices_np, backers_np, num_creator_projects

In [110]:
# Function to collect all features
def collect_data(start = 1, end = 2, category = 'music'):
    samples = []
    reward_prices = []
    count = (start - 1)*9
    for i in tqdm(range(start,end)):
        url_i = create_url(i,category)
        #print(url_i)
        #bs_i, ok = get_page(url_i)
        # Load the website by driver
        chrome_path = './chromedriver.exe'
        driver_i = webdriver.Chrome(chrome_path)
        driver_i.get(url_i)
        time.sleep(5)

        # Accept the cookie
        cookie_question_i = driver_i.find_element_by_xpath("//*[@id=\"App-react-component\"]/div[1]/div/div/div/div/button[2]")
        cookie_question_i.click()

        # Load the html page
        html_i = driver_i.page_source
        bs_i = BeautifulSoup(html_i, 'html.parser')
        
        projects = bs_i.findAll('a', 'styles__StyledCrowdfundingCard-sc-1dxuhb7-0 dOcwdr k-CrowdfundingCard k-Card k-Card--light k-Card--withoutBoxShadowOnHover k-CrowdfundingCard--titlesMinHeight')
        for p in tqdm(projects):
            # Extract is_successful
            is_successful = p.findAll('span','state-badge__StyledState-sc-1ixvpgd-0 efdutF')[0].contents[1]

            # Extract percentage_fund
            percentage_fund = p.findAll('span', 'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-color-font1 k-u-size-micro k-u-weight-regular k-CrowdfundingCard__progressBar__percent k-u-hidden@s-down')[0].contents[0]

            # Extract funded, target, in_2020, backers
            infos = p.findAll('div', 'k-CrowdfundingCard__informations__infoContainer')
            funded = infos[2].findAll('strong', 'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-weight-regular info__StyledText-h7ofxb-0 jfWZBp')[0].contents[0]
            target = infos[2].findAll('span', 'k-u-hidden@s-down')[0].contents[0]
            in_2020 = infos[1].findAll('strong', 'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-weight-regular info__StyledText-h7ofxb-0 jfWZBp')[0].contents[0]
            backers = p.findAll('strong', 'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-weight-regular info__StyledText-h7ofxb-0 jfWZBp')[0].contents[0]

            # Extract category

            # Extract thumbnail type
            img = p.findAll('div','k-CrowdfundingCard__image__imageContainer')[0]
            thumbnail_type = img.find('img').get('src').split('.')[-1]

            # Extract remaining features
            url_pro = p.get('href')
            num_news, num_comments, num_contributions, num_rewards, min_price, max_price, prices_np, backers_np, num_creator_projects = go_to_project(url_pro)

            print(num_news, num_comments, num_contributions,num_rewards, min_price, max_price)
            #print(backers_np)
            #print(prices_np)
            #print(num_creator_projects)
            #print('----------------------------------')
            sample_i = [
                count,
                is_successful,
                percentage_fund,
                funded,
                target,
                in_2020,
                backers,
                category,
                thumbnail_type,
                num_rewards,
                min_price,
                max_price,
                num_news,
                num_comments,
                num_contributions,
                num_creator_projects
            ]
            for j in range(len(prices_np)):
                rp = [count, category, prices_np[j], backers_np[j]]
                reward_prices.append(rp)
            samples.append(sample_i)
            count += 1
            time.sleep(random.random()*2)
        driver_i.quit()
        time.sleep(random.random()*5)
    return samples, reward_prices


### Tests

In [72]:
url = create_url(2,'music')
bs, ok = get_page(url)
url, ok

('https://www.kisskissbankbank.com/en/discover?project[successful]=on&categories[music]=on&page=2',
 True)

In [11]:
get_num_of_projects(bs)

5679

In [12]:
num_pages = get_num_of_pages(bs)
num_pages

632

### Main

In [60]:
features_data = [
    'id',
    'is_successful',
    'percentage_fund',
    'funded',
    'target',
    'in_2020',
    'backers',
    'category',
    'thumbnail_type',
    'num_rewards',
    'min_price',
    'max_price',
    'num_news',
    'num_comments',
    'num_contributions',
    'num_creator_projects'
]
features_rewards = [
    'id',
    'category',
    'price',
    'backers'
]

In [111]:
data, rewards = collect_data(10,25,'music')

  0%|          | 0/9 [00:00<?, ?it/s]

1 31 97 12 2.0 5000.0



 11%|█         | 1/9 [00:44<05:52, 44.12s/it]

2 23 86 4 30.0 200.0



 22%|██▏       | 2/9 [01:20<04:53, 41.94s/it]

...


 78%|███████▊  | 7/9 [04:32<01:17, 38.94s/it]

0 5 26 12 5.0 1000.0



 89%|████████▉ | 8/9 [05:08<00:37, 37.96s/it]

1 18 58 6 20.0 900.0



100%|██████████| 15/15 [1:30:03<00:00, 360.24s/it]


In [112]:
data_df = pd.DataFrame(data, columns = features_data)
data_df

,id,is_successful,percentage_fund,funded,target,in_2020,backers,category,thumbnail_type,num_rewards,min_price,max_price,num_news,num_comments,num_contributions,num_creator_projects
0,81,successful,114 %,"€5,679","Out of €5,000",8/5/2020,92,music,png,12,2.0,5000.0,1,31,97,1
1,82,successful,100 %,"€9,000","Out of €9,000",8/4/2020,80,music,jpg,4,30.0,200.0,2,23,86,1
2,83,successful,125 %,"€3,124","Out of €2,500",8/4/2020,124,music,jpg,9,6.0,800.0,0,25,126,1
3,84,successful,103 %,"€4,115","Out of €4,000",8/2/2020,73,music,jpg,7,5.0,200.0,0,14,79,1
4,85,successful,101 %,"€9,089","Out of €9,000",8/1/2020,76,music,JPG,12,20.0,1750.0,2,26,86,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,211,successful,111 %,"€4,440","Out of €4,000",2/17/2020,88,music,png,8,10.0,500.0,0,8,93,5
131,212,successful,113 %,"€4,520","Out of €4,000",2/16/2020,86,music,jpg,13,5.0,1000.0,3,23,88,1
132,213,successful,103 %,"€3,618","Out of €3,500",2/10/2020,37,music,png,6,3.0,500.0,0,4,40,1
133,214,successful,100 %,"€3,000","Out of €3,000",2/7/2020,20,music,png,12,5.0,1000.0,0,5,26,1


In [113]:
rewards_df_ = pd.DataFrame(rewards, columns = features_rewards)
rewards_df

,id,category,price,backers
0,0,music,15.0,25
1,0,music,25.0,42
2,0,music,35.0,25
3,0,music,55.0,19
4,0,music,80.0,5
...,...,...,...,...
877,98,music,1.0,1
878,98,music,10.0,6
879,98,music,30.0,8
880,98,music,50.0,1


In [114]:
data_df.to_csv('./data/data_music_2.csv', index=False)
rewards_df.to_csv('./data/rewards_music_2.csv', index=False)